### Canvas Events
Das Canvas-Widget erkennt, wenn auf die Leinwand geklickt wird.
Ist das Widget aktive, so erkennt es auch Tastendrücke. 
Mausklicks und Tastendrücke sind sog. Events.

Beim Eintreffen eines Events ruft das Widget dann die Funktion auf, die 
zur Handhabung diese Events registriert wurde. 
Eine solche Funktion nennt man auch Callback oder Eventhandler.

Das Widget reagiert auf die Events `on_key_down`, `on_mouse_down`, `on_mouse_up` und `on_mouse_out` (Mauszeiger verlässt Canvas-Widget). 
Für ein Canvas-Objekt `canvas` kann mit
```python
canvas.on_mouse_down(f)
```
eine Funktion `f` registiert werden, die
beim Eintreffen des Events `on_mouse_down` aufgerufen wird. 
Analog können Funktionen für die Events  `on_key_down`,`on_mouse_up` und `on_mouse_out` 
registriert werden.

Mit was für Argumenten die Callbacks aufgerufen werden ist oft 
etwas dürftig dokumentiert. Um das herauszufinden registriert man am besten 
als Callback eine Funktion, welche alle Argumente ausgibt, mit denen sie aufgerufen wurde. (siehe unten).

Wir werden sehen, dass `on_mouse_down` und `on_mouse_down` mit
2 Argumenten aufgerufen werden, der x- und y-Koordinate der Position.
`on_key_down` wird mit 4 Argumenten aufgerufen: Dem Tastenname und 3 Booleans, je eins für die Shift, Control und Windows Taste.

In [ ]:
from IPython.display import display  # sonst ist display (zu unrecht) mit einer roten Wellenlinie unterlegt#
from ipywidgets import Output


out = Output(layout={'border': '1px solid black'})

In [ ]:
@out.capture()
def f_keydown(*args):
    print('Event: on_key_down, args:', args)


@out.capture()
def f_mousedown(*args):
    print('Event: on_mouse_down, args:', args)


@out.capture()
def f_mouseup(*args):
    print('Event: on_mouse_up, args:', args)


@out.capture()
def f_mouseout(*args):
    print('Event: on_mouse_out, args:', args)

In [ ]:
from ipycanvas import Canvas


canvas = Canvas(width=300, height=200, layout={'border': '1px solid black'})

# Callbacks registrieren
canvas.on_key_down(f_keydown)
canvas.on_mouse_down(f_mousedown)
canvas.on_mouse_up(f_mouseup)
canvas.on_mouse_out(f_mouseout)

display(canvas, out)

In [ ]:
# Inhalt von out loeschen
out.clear_output()

***
Wir definieren nun Callbacks für die Events
`on_key_down`, `on_mouse_down` und `on_mouse_up`.
Diese Callbacks sollen folgendes tun:
```python
def on_key_down(key, *flags):
    '''schreibe den Namen der gedrückten Taste auf die Leinwand.
       Wird 'c' gedrückt lösche die Leinwand.
    '''
    

def on_mouse_down(x, y):
    '''speichere die Position (x,y)'''


def on_mouse_up(x, y):
    '''zeichne eine Linie von der Position, wo zuletzt gelickt wurde
       zur aktuellen Position
    '''
    
```

Ruft ein Jupyterlab-Widget eine Callback-Funktion auf,
so wird deren Output (**inkl. Fehlermeldugen**) nicht direkt ins 
Notebook geschreiben, sondern (wenn man Glück hat) in die Log-Console. 
Ruft ein Widget ein fehlerhaftes Callback auf, passiert oft einfach gar nichts, was das auffinden des Fehlers sehr schwierig macht.

Aus diesem Grund empfiehlt es sich,
den Output der Callbacks in ein Output-Widget umzuleiten.
***

In [ ]:
state = {'mouse_down': None}  # hier speichern wir die geklickte Position


def get_fontsize(canvas):
    i = canvas.font.index('px')
    return int(canvas.font[:i])


@out.capture()
def on_key_down(key, *flags):
    # print('Event: on_key_down, args:', key, *flags)
    if key == 'c':
        canvas.clear()
    font_size = get_fontsize(canvas)
    canvas.clear_rect(0, 0, canvas.width, 1.5*font_size)
    canvas.fill_text(key, 20, font_size)


@out.capture()
def on_mouse_down(x, y):
    # print('Event: on_mouse_down, args:', x, y)
    state['mouse_down'] = (x, y)
    canvas.fill_circle(x, y, 2)


@out.capture()
def on_mouse_up(x, y):
    # print('Event: on_mouse_up, args:', x, y)
    last_pos = state['mouse_down']
    if last_pos:
        canvas.stroke_lines([last_pos, (x, y)])
    state['mouse_down'] = None

In [ ]:
canvas = Canvas(width=300, height=200, layout={'border': '1px solid black'})


# line-width, stroke-color und font setzen.
canvas.font = '20px sans-serif'
canvas.stroke = 'blue'
canvas.line_width = 2

# Callbacks registrieren
canvas.on_key_down(on_key_down)
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_up(on_mouse_up)

display(canvas, out)  # Leinwand und Outputwidget anzeigen.